In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim # torch  선언
torch.manual_seed(1)

import random
random.seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
      torch.cuda.manual_seed_all(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sejongai19011494/train-1.csv
/kaggle/input/sejongai19011494/test-1.csv
/kaggle/input/sejongai19011494/submission.csv


In [2]:
X = pd.read_csv('/kaggle/input/sejongai19011494/train-1.csv')
y = X['건당이용금액']
test = pd.read_csv('/kaggle/input/sejongai19011494/test-1.csv')
X.head()

,기준년월,강수량,평균기온,최고기온,최저기온,제주 대분류,제주 중분류,업종명,성별,연령대별,카드이용건수,건당이용금액
0,2015-10,61.0,18.9,25.5,10.7,서귀포시,예래동,슈퍼 마켓,여,20대,30,10881
1,2015-09,172.9,23.2,28.6,18.4,제주시,연동,약국,여,30대,239,12597
2,2015-11,215.5,15.4,22.6,2.1,서귀포시,남원읍,슈퍼 마켓,여,40대,131,21265
3,2016-04,232.2,15.8,22.7,10.8,서귀포시,예래동,기념품 점,여,40대,29,17000
4,2016-04,121.4,15.6,24.5,9.8,제주시,연동,슈퍼 마켓,여,50대,369,22050


In [3]:
X['연도'] = X['기준년월'].str.split('-').str[0].astype(int)
X['월'] = X['기준년월'].str.split('-').str[1].astype(int)
X = X.drop([ '기준년월', '건당이용금액'], axis=1)

test['연도'] = test['기준년월'].str.split('-').str[0].astype(int)
test['월'] = test['기준년월'].str.split('-').str[1].astype(int)
test = test.drop(['기준년월'], axis=1)

In [4]:
list =['제주 대분류', '제주 중분류', '업종명', '성별', '연령대별']

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for c in list:
    label = pd.concat([X[c], test[c]], axis=0)
    le.fit(label)
    X[c] = le.transform(X[c])
    test[c] = le.transform(test[c])

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sc = MinMaxScaler()
X_sc = sc.fit_transform(X)
test_sc = sc.transform(test)

In [6]:
X = torch.FloatTensor(X.to_numpy()).to(device)
test = torch.FloatTensor(test.to_numpy()).to(device)
y = torch.FloatTensor(y.to_numpy()).to(device)

X_sc = torch.FloatTensor(X_sc).to(device)
test_sc = torch.FloatTensor(test_sc).to(device)

In [7]:
# DNN 모델 만들기
linear1 = torch.nn.Linear(12, 256)
linear2 = torch.nn.Linear(256, 128)
linear3 = torch.nn.Linear(128, 128)
linear4 = torch.nn.Linear(128, 1)
relu = torch.nn.ReLU() # relu 사용
dropout = torch.nn.Dropout(0.2)

# weight 초기화
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)

model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2,  relu, dropout,
                            linear3,  relu, dropout,
                            linear4).to(device)

In [8]:
# 모델 학습
model.train()
loss = torch.nn.L1Loss() 
optimizer = optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(10000):
    hypothesis = model(X_sc)
    cost = loss(hypothesis, y.unsqueeze(1))
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%1000==0:
        print(epoch, cost.item())

0 35100.23828125
1000 17965.34765625
2000 14471.115234375
3000 9598.62890625
4000 9040.890625
5000 8580.6455078125
6000 8350.7939453125
7000 7962.37060546875
8000 7775.63037109375
9000 7432.36376953125


In [9]:
model.eval()
with torch.no_grad():
    predict = model(test_sc).cpu()
    predict = predict.detach().numpy().squeeze(1)


In [10]:
submission = pd.read_csv('/kaggle/input/sejongai19011494/submission.csv')
submission['Predicted'] = predict

submission.to_csv("submission.csv", index=False)
submission.head()

,Id,Predicted
0,0,14562.234375
1,1,11846.511719
2,2,13646.103516
3,3,37549.078125
4,4,14229.337891
